# 本地大型语言模型 (LLM)

在这里，我们将使用一个为本地大型语言模型 (LLM) 提供服务的服务器。

**注意**：这适用于原型设计/开发用途，但当可能存在来自不同用户的并发请求时，不应用于生产环境。截至目前，Ollama 被设计为单用户使用，无法处理并发请求，详见此问题：[Parallel requests · Issue #358 · ollama/ollama](https://github.com/ollama/ollama/issues/358)。

In [7]:
# !pip install langchain langchain_community langserve

In [8]:
from langchain.prompts.chat import ChatPromptTemplate

In [9]:
from langserve import RemoteRunnable

model = RemoteRunnable("http://localhost:8000/ollama/")

让我们测试一下聊天模型的标准接口。

In [12]:
prompt = "请听我讲述一个关于猫咪的三句话故事。"

In [14]:
model.invoke(prompt)

AIMessage(content='等你准备好了，按你的要求开始讲。', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-04T05:35:46.345586Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3021375389, 'load_duration': 34721363, 'prompt_eval_count': 25, 'prompt_eval_duration': 229295000, 'eval_count': 13, 'eval_duration': 2755150000}, id='run-1bbabe1a-d0fb-41dd-ab32-6556bab4d227-0')

In [15]:
await model.ainvoke(prompt)

AIMessage(content='当然可以！你先说第一句。', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-04T05:35:50.165361Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2218906968, 'load_duration': 101420104, 'prompt_eval_count': 25, 'prompt_eval_duration': 232897000, 'eval_count': 10, 'eval_duration': 1882589000}, id='run-b0623861-7700-4536-9be3-d0e4f73bf10b-0')

批量API是有效的，但由于ollama不支持并行处理，它的速度并不比调用.invoke两次快。

In [16]:
%%time
model.batch([prompt, prompt])

CPU times: user 6.16 ms, sys: 2.79 ms, total: 8.95 ms
Wall time: 23.1 s


[AIMessage(content='好吧，请开始讲故事了！', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-04T05:52:52.23673Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 20209445573, 'load_duration': 11892913269, 'prompt_eval_count': 25, 'prompt_eval_duration': 3085068000, 'eval_count': 9, 'eval_duration': 5227245000}, id='run-b02192fd-645c-4c0e-8ac6-53120281861d-0'),
 AIMessage(content='当然可以！我很高兴地听你讲述这个关于猫咪的故事。', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-04T05:52:55.135358Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 23108231846, 'load_duration': 11894639816, 'prompt_eval_count': 25, 'prompt_eval_duration': 3020765000, 'eval_count': 21, 'eval_duration': 5105026000}, id='run-9962638d-5bf7-4161-ae2a-f4db255fcee0-0')]

In [17]:
%%time
for _ in range(2):
    model.invoke(prompt)

CPU times: user 6.84 ms, sys: 2.03 ms, total: 8.87 ms
Wall time: 8.22 s


In [18]:
await model.abatch([prompt, prompt])

[AIMessage(content='好的，请开始吧！', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-04T05:53:14.715299Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1934266873, 'load_duration': 31591576, 'prompt_eval_count': 25, 'prompt_eval_duration': 260860000, 'eval_count': 6, 'eval_duration': 1433818000}, id='run-7fc9f68a-a995-42cc-bf3a-8500b16ba361-0'),
 AIMessage(content='好的，请发表您的故事！', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-04T05:53:14.931201Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2150361879, 'load_duration': 30730413, 'prompt_eval_count': 25, 'prompt_eval_duration': 205887000, 'eval_count': 8, 'eval_duration': 1910737000}, id='run-596b6060-9719-418d-8948-acd7ef4fd17c-0')]

流式传输默认可用。

In [19]:
for chunk in model.stream(prompt):
    print(chunk.content, end="|", flush=True)

好|啊|！|我|准备|好了|，你|可以|开始|讲|故事|了|！||

In [20]:
async for chunk in model.astream(prompt):
    print(chunk.content, end="|", flush=True)

当然|，可以|听|你|讲|述|这个|故事|了|。||

事件流API同样可用。

In [21]:
i = 0
async for event in model.astream_events(prompt, version='v1'):
    print(event)
    if i > 10:
        print('...')
        break
    i += 1

{'event': 'on_chat_model_start', 'run_id': '7ab179d9-f924-4fa4-8561-c242307b5e32', 'name': '/ollama', 'tags': [], 'metadata': {}, 'data': {'input': '请听我讲述一个关于猫咪的三句话故事。'}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '7ab179d9-f924-4fa4-8561-c242307b5e32', 'tags': [], 'metadata': {}, 'name': '/ollama', 'data': {'chunk': AIMessageChunk(content='好', id='run-7ab179d9-f924-4fa4-8561-c242307b5e32')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '7ab179d9-f924-4fa4-8561-c242307b5e32', 'tags': [], 'metadata': {}, 'name': '/ollama', 'data': {'chunk': AIMessageChunk(content='啊', id='run-7ab179d9-f924-4fa4-8561-c242307b5e32')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '7ab179d9-f924-4fa4-8561-c242307b5e32', 'tags': [], 'metadata': {}, 'name': '/ollama', 'data': {'chunk': AIMessageChunk(content='，我', id='run-7ab179d9-f924-4fa4-8561-c242307b5e32')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '7ab179d9-f924-4fa4-8561-c242307b5